# Lesson 3: Reflection and Blogpost Writing

## Setup

In [3]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [4]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [5]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 02-10 20:21:31] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [7]:
print(reply)

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its limitless potential with DeepLearning.AI. Offering top-notch courses created by industry expert Andrew Ng, this platform equips learners with the skills needed to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all in a user-friendly and engaging manner. Elevate your knowledge and stay ahead in this rapidly evolving field. Whether you're a beginner or an experienced professional, DeepLearning.AI provides the resources to help you thrive. Join the community today and embark on a journey towards AI mastery.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [8]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 02-10 20:26:38] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its limitless potential with DeepLearning.AI. Offering top-notch courses created by industry expert Andrew Ng, this platform equips learners with the skills needed to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all in a user-friendly and engaging manner. Elevate your knowledge and stay ahead in this rapidly evolving field. Whether you're a beginner or an experienced professional, DeepLearning.AI provides the resources to help you thrive. Join the community today and embark on a journey towards AI mastery.

---------------------------------------------

## Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


[autogen.oai.client: 02-10 20:28:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 02-10 20:28:08] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 02-10 20:28:10] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 02-10 20:28:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Orchestrate the nested chats to solve the task

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its limitless potential with DeepLearning.AI. Offering top-notch courses created by industry expert Andrew Ng, this platform equips learners with the skills needed to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all in a user-friendly and engaging manner. Elevate your knowledge and stay ahead in this rapidly evolving field. Whether you're a beginner or an experienced professional, DeepLearning.AI provides the resources to help you thrive. Join the community today and embark on a journey towards AI mastery.

---------------------------------------------

## Get the summary

In [17]:
print(res.summary)

Title: Mastering Deep Learning: A Comprehensive Guide to DeepLearning.AI

Embark on a journey of artificial intelligence mastery with DeepLearning.AI by Andrew Ng. Explore cutting-edge courses in deep learning, computer vision, and natural language processing. Elevate your skills and stay ahead in this rapidly evolving field. From beginners to experts, this platform offers something for everyone. Dive into the world of AI with industry-leading courses that are both user-friendly and engaging. Join a thriving community and unleash the power of deep learning today.

Incorporate primary keywords related to deep learning, artificial intelligence, and online courses for SEO optimization. Craft a meta title and description reflective of the content while enhancing with keywords. Utilize internal linking for seamless user navigation. Disclose any affiliations with DeepLearning.AI transparently and ensure accuracy in course offerings. Respect user privacy by clearly defining data practices.
